# Introducción a los Tipos Abstractos de Datos (TAD)

## Concepto de TAD

Un Tipo Abstracto de Datos (TAD) es una especificación de un conjunto de datos y las operaciones que se pueden realizar con esos datos. El TAD se centra en qué operaciones existen y no en cómo se implementan. Es una herramienta conceptual que nos permite abstraernos de los detalles y pensar en términos de operaciones y datos.

### Ejemplo de TAD: Lista


In [1]:
lista = []

def agregar(elemento):
    lista.append(elemento)

def eliminar(elemento):
    if elemento in lista:
        lista.remove(elemento)

def buscar(elemento):
    return elemento in lista

## Desventajas de este enfoque

1. **Falta de encapsulamiento**: En el ejemplo proporcionado, tanto las operaciones como los datos están expuestos y pueden ser modificados directamente desde fuera del TAD. Esto viola el principio de encapsulamiento, que es fundamental para el diseño de software robusto y mantenible.

2. **Dificultad en reutilización**: Con el diseño actual, si quisieras tener múltiples listas y aplicar operaciones sobre ellas, tendrías que crear funciones separadas o modificar las existentes, lo que no es práctico.

3. **Problemas de extensibilidad**: Cambiar la implementación interna (por ejemplo, cambiar de una lista a un árbol binario para mejorar la eficiencia) requeriría modificar todas las funciones que interactúan con la estructura de datos. Esto hace que el código sea difícil de mantener y de extender con nuevas funcionalidades.

4. **Falta de abstracción**: Aunque el TAD debería ayudarnos a abstraernos de los detalles de implementación, el enfoque actual no lo hace de manera efectiva. Esto podría llevar a errores si no se comprenden completamente las implementaciones subyacentes.


## Desafíos

### Desafío 1: Sistemas con Múltiples Entidades Interconectadas
Imagina un sistema de gestión de biblioteca que maneja libros, usuarios, préstamos y multas. Usar TADs separados para cada uno de estos elementos podría complicar la interacción y gestión de relaciones entre ellos.


In [ ]:
from datetime import date, timedelta

# TAD Libro
class Libro:
    def __init__(self, codigo, titulo, autor):
        self.codigo = codigo
        self.titulo = titulo
        self.autor = autor
        self.disponible = True

    def prestar(self):
        if self.disponible:
            self.disponible = False
            return True
        return False

    def devolver(self):
        self.disponible = True

    def __str__(self):
        estado = "Disponible" if self.disponible else "Prestado"
        return f"{self.titulo} ({self.autor}) - {estado}"


# TAD Usuario
class Usuario:
    def __init__(self, nombre):
        self.nombre = nombre
        self.prestamos = []
        self.multas = 0

    def agregar_prestamo(self, prestamo):
        self.prestamos.append(prestamo)

    def agregar_multa(self, monto):
        self.multas += monto

    def __str__(self):
        return f"Usuario: {self.nombre} | Multas: ${self.multas}"


# TAD Prestamo
class Prestamo:
    def __init__(self, usuario, libro, dias_prestamo=7):
        self.usuario = usuario
        self.libro = libro
        self.fecha_inicio = date.today()
        self.fecha_vencimiento = self.fecha_inicio + timedelta(days=dias_prestamo)
        self.devuelto = False

    def devolver(self):
        self.devuelto = True
        self.libro.devolver()

    def esta_vencido(self):
        return date.today() > self.fecha_vencimiento and not self.devuelto

    def __str__(self):
        estado = "Devuelto" if self.devuelto else "En préstamo"
        return f"{self.libro.titulo} → {self.usuario.nombre} ({estado})"


# TAD Biblioteca (coordina todo)
class Biblioteca:
    def __init__(self):
        self.libros = []
        self.usuarios = []
        self.prestamos = []

    def agregar_libro(self, libro):
        self.libros.append(libro)

    def registrar_usuario(self, usuario):
        self.usuarios.append(usuario)

    def prestar_libro(self, usuario, libro):
        if libro.prestar():
            prestamo = Prestamo(usuario, libro)
            usuario.agregar_prestamo(prestamo)
            self.prestamos.append(prestamo)
            print(f" {usuario.nombre} ha tomado prestado '{libro.titulo}'.")
        else:
            print(f" El libro '{libro.titulo}' no está disponible.")

    def devolver_libro(self, usuario, libro):
        for prestamo in usuario.prestamos:
            if prestamo.libro == libro and not prestamo.devuelto:
                prestamo.devolver()
                print(f" {usuario.nombre} devolvió '{libro.titulo}'.")
                # Se aplica multa si está vencido
                if prestamo.esta_vencido():
                    usuario.agregar_multa(100)
                    print(f"️ Multa aplicada a {usuario.nombre} por retraso ($100).")
                return
        print(f"️ No se encontró préstamo activo de '{libro.titulo}' para {usuario.nombre}.")

    def mostrar_estado(self):
        print("\n Estado de la Biblioteca:")
        for libro in self.libros:
            print("  -", libro)
        print("\n Usuarios:")
        for usuario in self.usuarios:
            print("  -", usuario)
        print("\n Préstamos:")
        for prestamo in self.prestamos:
            print("  -", prestamo)


# Ejemplo:
if __name__ == "__main__":
    # Se crea la biblioteca
    biblioteca = Biblioteca()

    # Se agregan libros
    libro1 = Libro(1, "Cien años de soledad", "G. García Márquez")
    libro2 = Libro(2, "El Principito", "Antoine de Saint-Exupéry")
    biblioteca.agregar_libro(libro1)
    biblioteca.agregar_libro(libro2)

    # Se registran los usuarios
    usuario1 = Usuario("Ana")
    usuario2 = Usuario("Luis")
    biblioteca.registrar_usuario(usuario1)
    biblioteca.registrar_usuario(usuario2)

    # Se realizan préstamos
    biblioteca.prestar_libro(usuario1, libro1)
    biblioteca.prestar_libro(usuario2, libro1)  # no disponible
    biblioteca.prestar_libro(usuario2, libro2)

    # Cuando se devulve un libro
    biblioteca.devolver_libro(usuario1, libro1)

    # Se muestra el estado final
    biblioteca.mostrar_estado()

Se comienza identificando las entidades principales del sistema de biblioteca (libros, usuarios, préstamos y la propia biblioteca), y se crea un TAD para cada una, representado mediante clases. Cada clase define sus atributos y métodos que encapsulan su comportamiento. Por ejemplo, la clase Libro maneja su disponibilidad, la clase Usuario registra los préstamos y multas, y la clase Prestamo gestiona las fechas y el estado de devolución. Luego, se diseña la clase Biblioteca como coordinadora que administra la interacción entre los distintos TAD: registrar usuarios, agregar libros, prestar y devolver ejemplares. Finalmente, se implementa un bloque principal que crea objetos, realiza operaciones y muestra el estado del sistema.


### Desafío 2: Cambio Frecuente en Requisitos
Supón que estás desarrollando un juego de video con distintos tipos de personajes y armas. Los requerimientos cambian con frecuencia, añadiendo nuevos personajes y habilidades. Mantener y actualizar TADs en este escenario podría ser una tarea titánica.


In [ ]:
# TAD base para personajes
class Personaje:
    def __init__(self, nombre, vida):
        self.nombre = nombre
        self.vida = vida
        self.arma = None

    def equipar_arma(self, arma):
        self.arma = arma

    def atacar(self, enemigo):
        if self.arma:
            danio = self.arma.usar()
            enemigo.recibir_danio(danio)
            print(f"{self.nombre} ataca a {enemigo.nombre} con {self.arma.nombre} causando {danio} puntos de daño.")
        else:
            print(f"{self.nombre} no tiene un arma equipada.")

    def recibir_danio(self, cantidad):
        self.vida -= cantidad
        if self.vida <= 0:
            print(f"{self.nombre} ha sido derrotado.")
        else:
            print(f"{self.nombre} tiene {self.vida} puntos de vida restantes.")


# TAD base para armas
class Arma:
    def __init__(self, nombre, danio):
        self.nombre = nombre
        self.danio = danio

    def usar(self):
        return self.danio


# Subclases que amplían el comportamiento
class Espada(Arma):
    def usar(self):
        print(f"La espada {self.nombre} corta con precisión.")
        return self.danio


class Arco(Arma):
    def usar(self):
        print(f"El arco {self.nombre} dispara una flecha veloz.")
        return self.danio - 1


class Hechizo(Arma):
    def usar(self):
        print(f"El hechizo {self.nombre} libera energía mágica.")
        return self.danio + 2


# Nuevos personajes (se pueden agregar fácilmente)
class Guerrero(Personaje):
    pass

class Mago(Personaje):
    def atacar(self, enemigo):
        print(f"{self.nombre} conjura un hechizo...")
        super().atacar(enemigo)


# Ejemplo:
if __name__ == "__main__":
    # Se crean personajes
    heroe = Guerrero("Arthas", 30)
    enemigo = Mago("Medivh", 25)

    # Se crean y se equipan armas
    espada = Espada("Justicia de Acero", 8)
    hechizo = Hechizo("Fuego Arcano", 6)

    heroe.equipar_arma(espada)
    enemigo.equipar_arma(hechizo)

    # Combate
    heroe.atacar(enemigo)
    enemigo.atacar(heroe)

Se comienza definiendo las entidades principales del videojuego (personajes y armas) y se representan mediante TAD en forma de clases. Primero se crean las clases base Personaje y Arma, que contienen los atributos y métodos comunes a todos los tipos. Luego se aplican los principios de herencia y polimorfismo para extender estas clases y crear subclases específicas como Guerrero, Mago, Espada, Arco y Hechizo, cada una con comportamientos personalizados. De esta manera, el sistema puede adaptarse fácilmente a nuevos requisitos, permitiendo agregar personajes o armas sin modificar el código existente. Finalmente, se implementa un bloque principal para crear objetos, equipar armas y simular un combate.


### Desafío 3: Estructuras de Datos Anidadas
Considera un sistema de manejo de inventario para una cadena de tiendas minoristas. Tienes que tratar con datos de productos, tiendas, empleados, y transacciones, donde cada tienda podría tener múltiples productos y empleados. Gestionar estas relaciones con TADs podría ser ineficiente y propenso a errores.


In [ ]:
# TAD Producto
class Producto:
    def __init__(self, nombre, precio, cantidad):
        self.nombre = nombre
        self.precio = precio
        self.cantidad = cantidad

    def vender(self, unidades):
        if unidades <= self.cantidad:
            self.cantidad -= unidades
            return True
        return False

    def __str__(self):
        return f"{self.nombre} - ${self.precio} ({self.cantidad} unidades)"


# TAD Empleado
class Empleado:
    def __init__(self, nombre, cargo):
        self.nombre = nombre
        self.cargo = cargo

    def __str__(self):
        return f"{self.nombre} ({self.cargo})"


# TAD Transaccion
class Transaccion:
    def __init__(self, producto, unidades, total):
        self.producto = producto
        self.unidades = unidades
        self.total = total

    def __str__(self):
        return f"Venta: {self.producto.nombre} x{self.unidades} = ${self.total}"


# TAD Tienda
class Tienda:
    def __init__(self, nombre):
        self.nombre = nombre
        self.productos = []
        self.empleados = []
        self.transacciones = []

    def agregar_producto(self, producto):
        self.productos.append(producto)

    def contratar_empleado(self, empleado):
        self.empleados.append(empleado)

    def vender_producto(self, nombre_producto, unidades):
        for producto in self.productos:
            if producto.nombre == nombre_producto and producto.vender(unidades):
                total = producto.precio * unidades
                transaccion = Transaccion(producto, unidades, total)
                self.transacciones.append(transaccion)
                print(f" Venta realizada en {self.nombre}: {transaccion}")
                return
        print(f" No se pudo completar la venta de {nombre_producto} en {self.nombre}.")

    def mostrar_estado(self):
        print(f"\n Tienda: {self.nombre}")
        print("Productos:")
        for p in self.productos:
            print("  -", p)
        print("Empleados:")
        for e in self.empleados:
            print("  -", e)
        print("Transacciones:")
        for t in self.transacciones:
            print("  -", t)


# TAD Cadena de Tiendas (nivel superior)
class CadenaTiendas:
    def __init__(self, nombre):
        self.nombre = nombre
        self.tiendas = []

    def agregar_tienda(self, tienda):
        self.tiendas.append(tienda)

    def mostrar_inventario_general(self):
        print(f"\n Inventario general de la cadena '{self.nombre}':")
        for tienda in self.tiendas:
            tienda.mostrar_estado()


# Ejemplo:
if __name__ == "__main__":
    # Se crea cadena
    cadena = CadenaTiendas("MegaStore")

    # Se crean tiendas
    tienda1 = Tienda("Sucursal Centro")
    tienda2 = Tienda("Sucursal Norte")

    # Se agregan productos
    tienda1.agregar_producto(Producto("Laptop", 1200, 10))
    tienda1.agregar_producto(Producto("Mouse", 25, 50))
    tienda2.agregar_producto(Producto("Celular", 900, 15))
    tienda2.agregar_producto(Producto("Auriculares", 50, 40))

    # Se cuentan empleados
    tienda1.contratar_empleado(Empleado("Ana", "Vendedora"))
    tienda2.contratar_empleado(Empleado("Luis", "Cajero"))

    # Se registran ventas
    tienda1.vender_producto("Laptop", 2)
    tienda2.vender_producto("Auriculares", 5)

    # Se agregan tiendas a la cadena
    cadena.agregar_tienda(tienda1)
    cadena.agregar_tienda(tienda2)

    # Se muestra estado completo
    cadena.mostrar_inventario_general()

Primero, se analizan las entidades principales del sistema de inventario (productos, empleados, transacciones y tiendas) y se crean TAD independientes para cada una mediante clases. Luego, se aplica el principio de composición, haciendo que una clase contenga instancias de otras (por ejemplo, una Tienda contiene productos, empleados y, transacciones). Finalmente, se crea una clase superior CadenaTiendas que agrupa varias tiendas, permitiendo gestionar toda la estructura de forma ordenada y modular.
